# ChatGPT Plugins

This example shows how to use ChatGPT Plugins within LangChain abstractions.

Note 1: This currently only works for plugins with no auth.

Note 2: There are almost certainly other ways to do this, this is just a first pass. If you have better ideas, please open a PR!

In [34]:
#!/usr/bin/env python3
from typing import List

from langchain.chat_models import ChatOpenAI
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools import AIPluginTool
from langchain.utilities import SerpAPIWrapper
from dotenv import load_dotenv
import openai
# Load default environment variables (.env)
load_dotenv()

True

In [15]:
def openai_call(
    prompt: str,
    model: str = 'gpt-4',
    temperature: float = 0.5,
    max_tokens: int = 100,
):
    while True:
        try:
            if model.startswith("llama"):
                # Spawn a subprocess to run llama.cpp
                cmd = cmd = ["llama/main", "-p", prompt]
                result = subprocess.run(cmd, shell=True, stderr=subprocess.DEVNULL, stdout=subprocess.PIPE, text=True)
                return result.stdout.strip()
            elif not model.startswith("gpt-"):
                # Use completion API
                response = openai.Completion.create(
                    engine=model,
                    prompt=prompt,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    top_p=1,
                    frequency_penalty=0,
                    presence_penalty=0,
                )
                return response.choices[0].text.strip()
            else:
                # Use chat completion API
                messages = [{"role": "user", "content": prompt}]
                response = openai.ChatCompletion.create(
                    model=model,
                    messages=messages,
                    temperature=temperature,
                    max_tokens=max_tokens,
                    n=1,
                    stop=None,
                )
                return response.choices[0].message.content.strip()
        except openai.error.RateLimitError:
            print(
                "The OpenAI API rate limit has been exceeded. Waiting 10 seconds and trying again."
            )
            time.sleep(10)  # Wait 10 seconds and try again
        else:
            break

In [24]:
def decompose_objective(objective: str):
    prompt = (
        f'Write me a painfully detailed discretized list of tasks for the following objective:'
        f'Objective: {objective}'
        f'Tasks should be simple and self-contained enough that it only takes one action to complete. List the tool you would use in parentheses after the task. An example of an ultimately discretized task that takes one action to complete would be "Purchase a plane ticket using Kayak (Kayak)" because that requires one API call or action from a user. An an example of a task that is too generic, is one that would need to be broken down further, like "plan a birthday party" because that involves picking a date (a discretized task), inviting people via email (a discretized task), picking a theme etc. '
        f'Tools you have access to: WolframAlpha (useful for computation), Ask the Human (ask the user for things like dates, preferences), the internet (for answers to current events), OpenTable (for reserving and finding restaurants), Kayak (for planning and finding flights), ChatGPT as a tool (for natural language reasoning, generating packing lists, grocery lists from recipes, code snippets, poems/"copy" and more), Shop - the web app (for purchasing goods like clothing, party supplies), Instacart (for purchasing food and drinks), Zapier (for sending emails, creating excel databases and more).'
    )
    print(openai_call(prompt, 'gpt-4', temperature=0.5, max_tokens=7500))
    # llm = ChatOpenAI(temperature=0.5)
    # tools = []
    # tools = load_tools(["requests"])
    # tools += [klarna_tool]

    # agent_chain = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
    # agent_chain.run(prompt)
    # task_completed = "Task completed!" in response
    # if task_completed:
    #     print(f"Completed the following task: {task}")
    # else:
    #     print(f"Decomposing the following task further: {task}")

In [26]:
def decompose_objective(objective: str):
    prompt = (
        f'Write me a painfully detailed discretized list of tasks for the following objective:'
        f'Objective: {objective}'
        f'Tasks should be simple and self-contained enough that it only takes one action to complete. List the tool you would use in parentheses after the task. An example of an ultimately discretized task that takes one action to complete would be "Purchase a plane ticket using Kayak (Kayak)" because that requires one API call or action from a user. An an example of a task that is too generic, is one that would need to be broken down further, like "plan a birthday party" because that involves picking a date (a discretized task), inviting people via email (a discretized task), picking a theme etc. '
        f'Tools you have access to: WolframAlpha (useful for computation), Ask the Human (ask the user for things like dates, preferences), the internet (for answers to current events), OpenTable (for reserving and finding restaurants), Kayak (for planning and finding flights), ChatGPT as a tool (for natural language reasoning, generating packing lists, grocery lists from recipes, code snippets, poems/"copy" and more), Shop - the web app (for purchasing goods like clothing, party supplies), Instacart (for purchasing food and drinks), Zapier (for sending emails, creating excel databases and more).'
    )
    response = openai_call(prompt, 'gpt-4', temperature=0.5, max_tokens=7500)
    return response.split('\n')

In [30]:
def preprocess_task(sentence):
    # Find the last set of parentheses in the sentence
    start_index = sentence.rfind("(")
    end_index = sentence.rfind(")")
    
    # If there are no parentheses in the sentence, return the original sentence
    if start_index == -1 or end_index == -1:
        return sentence
    
    # Replace the last set of parentheses with "using" and the text within the parentheses
    using_text = sentence[start_index+1:end_index]
    new_sentence = sentence[:start_index] + "using " + using_text + sentence[end_index+1:]
    return new_sentence, using_text

tool_map = {
    'Ask the Human': 'human',
    'the internet': 'serpapi',
    'Kayak': AIPluginTool.from_plugin_url("https://www.kayak.com/.well-known/ai-plugin.json"),
    'Instacart': AIPluginTool.from_plugin_url("https://www.klarna.com/.well-known/ai-plugin.json"),
    # 'Klarna': klarna_tool = AIPluginTool.from_plugin_url("https://www.klarna.com/.well-known/ai-plugin.json")
}

def accomplish_task(task):
    task, tool_name = preprocess_task(task)
    llm = ChatOpenAI(temperature=0.5)
    tools = load_tools(["requests"])
    if tool_name == 'ChatGPT':
        print(openai_call(prompt, 'gpt-4', temperature=0.5, max_tokens=7500))
    else:
        tool = tool_map.get(tool_name, None)
        if tool is not None:
            tools += [tool]
        agent_chain = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
        agent_chain.run(prompt)

In [32]:
tasks = decompose_objective('Plan a vacation.')
print(tasks)

1. Ask the Human for preferred travel dates (Ask the Human)
2. Ask the Human for destination preferences (Ask the Human)
3. Research top vacation destinations based on preferences (the internet)
4. Present top 3 vacation destinations to the Human (ChatGPT)
5. Ask the Human to choose a destination from the list (Ask the Human)
6. Research local attractions and activities at the chosen destination (the internet)
7. Present top 5 attractions and activities to the Human (ChatGPT)
8. Ask the Human for their preferred attractions and activities (Ask the Human)
9. Generate a list of attractions and activities based on the Human's preferences (ChatGPT)
10. Search for flights to the chosen destination on the preferred travel dates (Kayak)
11. Present the top 3 flight options to the Human (ChatGPT)
12. Ask the Human to choose a flight from the list (Ask the Human)
13. Purchase the chosen flight using Kayak (Kayak)
14. Search for accommodation options at the chosen destination (the internet)
15. 

In [ ]:
for task in tasks:
    accomplish_task(task)